<a href="https://colab.research.google.com/github/stoictrader/LSTM_categorical/blob/master/sentiment_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/drive/') 



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#!unzip -uq "drive/My Drive/deeplearning/trainingandtestdata.zip" -d "drive/My Drive/deeplearning/tweet140"
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re


data = pd.read_csv('/content/drive/My Drive/deeplearning/tweet140/tweet_sentiment.csv',encoding='ISO-8859-1')
data.columns = ['sentiment','string_id','date','query','handle','text']
data = data[['sentiment','text']]
#data = data[data.sentiment != "Neutral"]
print('before randomize ', '\n',data['text'].head(5))



before randomize  
 0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 
Name: text, dtype: object


In [0]:
"remove @mentions"
data = data.replace("\@[a-zA-Z]+", "",regex=True)

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print((data['sentiment']==4).sum())
print((data['sentiment']==0).sum())


"randomize data"
data = data.sample(frac=1).reset_index(drop=True)
print('after randomize, lower case, removing mentions ', '\n',data['text'].head(5))



800000
799999
after randomize, lower case, removing mentions  
 0     is am addicted to lolcats thx to  im sending ...
1    i hate weeding shower picking up julia then wa...
2     just remember to take off your bra during thu...
3     apparently we dont have to do another mpn it ...
4    this is not what a summer schedule is supposed...
Name: text, dtype: object


In [0]:
max_features = 2000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)





In [0]:
#......................................................... lstm .......
def get_model(X):
  lstm_out = 196
  embed_dim = 128
  model = Sequential()
  model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
  model.add(SpatialDropout1D(0.4))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  print(model.summary())
  return(model)

model = get_model(X=X)


In [0]:
"This will convert positive and negative sentiment numbers"
Y =  pd.get_dummies(data['sentiment']).values
Y[0]
data.iloc[0]['sentiment']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.50, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(799999, 40) (799999, 2)
(800000, 40) (800000, 2)


In [0]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpointer = (ModelCheckpoint(filepath="/content/drive/My Drive/deeplearning/nlp.h5", monitor='val_accuracy',
                                verbose=1, save_best_only=True))

earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='max')

k=0
trainac=[]
valac=[]
#class_weights={0:1.,1:1.}
batch_size = 332
history = model.fit(X_train, Y_train, epochs = 30, batch_size=batch_size, verbose = 2,
                    
          validation_data=(X_test,Y_test),callbacks=[earlystopper,checkpointer])


#epochs = 155 best trained afterwords overfit expect 65% accurecy

trainl=(history.history['acc']) #, label='train')
vall=(history.history['val_acc'])

df_fit=pd.DataFrame({'trainac':trainl,'valac':vall})
df_fit.plot()


In [0]:
"retrain from where you left if colab disconnects after 12 hr"
new_model = get_model(X=X)

from tensorflow.keras.models import load_model
model_path = "/content/drive/My Drive/deeplearning/nlp.h5"
n_model = load_model(model_path)
old_weights  = n_model.get_weights()
new_model.set_weights(old_weights)

batch_size = 332
history = new_model.fit(X_train, Y_train, epochs = 30, batch_size=batch_size, verbose = 2,
          validation_data=(X_test,Y_test),callbacks=[earlystopper,checkpointer])


#epochs = 155 best trained afterwords overfit expect 65% accurecy

trainl=(history.history['acc']) #, label='train')
vall=(history.history['val_acc'])

df_fit=pd.DataFrame({'trainac':trainl,'valac':vall})
df_fit.plot()
